In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch import nn
from itertools import product

# Load data
df_train = pd.read_csv("data/final_data_with_patterns.csv")
df_test = pd.read_csv("test_data/final_test_data_with_patterns.csv")
feature_cols = [c for c in df_train.columns if c not in ["timestamp", "label"]]

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(df_train["label"])
y_test_raw = df_test["label"]
mask_known = y_test_raw.isin(le.classes_)
df_test = df_test[mask_known].copy()
y_test = le.transform(df_test["label"])

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(df_train[feature_cols])
X_test = scaler.transform(df_test[feature_cols])

# Sequence builder
def create_sequences(X, y, seq_len):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len + 1):
        x_win = X[i:i+seq_len]
        y_win = y[i:i+seq_len]
        if np.all(y_win == y_win[0]):
            X_seq.append(x_win)
            y_seq.append(y_win[0])
    return np.array(X_seq), np.array(y_seq)

# LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# Grid configuration
hidden_sizes = [32, 64, 128, 256]
seq_lens = [2, 5, 10, 20]
batch_sizes = [16, 32, 64, 128]
learning_rates = [0.01, 0.02, 0.001, 0.005]

grid = list(product(hidden_sizes, seq_lens, batch_sizes, learning_rates))
print(f"Starting Grid Search: {len(grid)} combinations...\n")

# Tracking
best_acc = 0.0
best_config = None
results = []

# Grid search loop
for i, (hidden_size, seq_len, batch_size, lr) in enumerate(grid, 1):
    print(f" Run {i}/{len(grid)} - hidden_size={hidden_size}, seq_len={seq_len}, batch_size={batch_size}, lr={lr}")

    # Create sequences
    X_tr_seq, y_tr_seq = create_sequences(X_train, y_train, seq_len)
    X_te_seq, y_te_seq = create_sequences(X_test, y_test, seq_len)

    # Convert to tensors
    X_tr = torch.tensor(X_tr_seq, dtype=torch.float32)
    y_tr = torch.tensor(y_tr_seq, dtype=torch.long)
    X_te = torch.tensor(X_te_seq, dtype=torch.float32)
    y_te = torch.tensor(y_te_seq, dtype=torch.long)

    train_loader = DataLoader(TensorDataset(X_tr, y_tr), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_te, y_te), batch_size=batch_size)

    # Model setup
    model = LSTMClassifier(input_size=len(feature_cols), hidden_size=hidden_size, num_classes=len(le.classes_))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(10):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            output = model(xb)
            loss = criterion(output, yb)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for xb, yb in test_loader:
            pred = model(xb)
            pred_class = torch.argmax(pred, dim=1)
            correct += (pred_class == yb).sum().item()
            total += yb.size(0)

    acc = correct / total
    results.append((hidden_size, seq_len, batch_size, lr, acc))

    if acc > best_acc:
        best_acc = acc
        best_config = (hidden_size, seq_len, batch_size, lr)

    print(f" Accuracy: {acc:.4f}\n")

# Best overall
print(" Best Configuration:")
print(f"  hidden_size={best_config[0]}, seq_len={best_config[1]}, batch_size={best_config[2]}, lr={best_config[3]}")
print(f"  Accuracy: {best_acc:.4f}")

# All results
print("\n All Results:")
for hs, sl, bs, lr_val, a in results:
    print(f"  hs={hs}, sl={sl}, bs={bs}, lr={lr_val} → acc={a:.4f}")

# Find optimal minimal config with acc = 1.0
perfect_configs = [r for r in results if r[4] == 1.0]
if perfect_configs:
    optimal = sorted(
        perfect_configs,
        key=lambda x: (x[0], x[1], x[2], -x[3])  # min hidden, min seq, min batch, max lr
    )[0]

    print("\n Optimal Configuration with 100% Accuracy (Minimal Resources):")
    print(f"  hidden_size={optimal[0]}, seq_len={optimal[1]}, batch_size={optimal[2]}, lr={optimal[3]}")
else:
    print("\n No configuration achieved 100% accuracy.")


🔍 Starting Grid Search: 256 combinations...

▶️ Run 1/256 - hidden_size=32, seq_len=2, batch_size=16, lr=0.01
✅ Accuracy: 0.9908

▶️ Run 2/256 - hidden_size=32, seq_len=2, batch_size=16, lr=0.02
✅ Accuracy: 0.9934

▶️ Run 3/256 - hidden_size=32, seq_len=2, batch_size=16, lr=0.001
✅ Accuracy: 0.9882

▶️ Run 4/256 - hidden_size=32, seq_len=2, batch_size=16, lr=0.005
✅ Accuracy: 0.9974

▶️ Run 5/256 - hidden_size=32, seq_len=2, batch_size=32, lr=0.01
✅ Accuracy: 0.9974

▶️ Run 6/256 - hidden_size=32, seq_len=2, batch_size=32, lr=0.02
✅ Accuracy: 0.9974

▶️ Run 7/256 - hidden_size=32, seq_len=2, batch_size=32, lr=0.001
✅ Accuracy: 0.9908

▶️ Run 8/256 - hidden_size=32, seq_len=2, batch_size=32, lr=0.005
✅ Accuracy: 0.9934

▶️ Run 9/256 - hidden_size=32, seq_len=2, batch_size=64, lr=0.01
✅ Accuracy: 0.9987

▶️ Run 10/256 - hidden_size=32, seq_len=2, batch_size=64, lr=0.02
✅ Accuracy: 0.9974

▶️ Run 11/256 - hidden_size=32, seq_len=2, batch_size=64, lr=0.001
✅ Accuracy: 0.9830

▶️ Run 12/256

In [14]:
# Recalculate computationally optimal config from saved results
perfect_configs = [r for r in results if r[4] == 1.0]

if perfect_configs:
    optimal = sorted(
        perfect_configs,
        key=lambda x: (x[0], x[1], x[2], -x[3])
    )[0]

    print("\n Optimal Configuration with 100% Accuracy (Least Cost):")
    print(f"  hidden_size = {optimal[0]}")
    print(f"  seq_len     = {optimal[1]}")
    print(f"  batch_size  = {optimal[2]}")
    print(f"  learning_rate = {optimal[3]}")
else:
    print("\n⚠ No configuration achieved 100% accuracy.")



 Optimal Configuration with 100% Accuracy (Least Cost):
  hidden_size = 32
  seq_len     = 5
  batch_size  = 16
  learning_rate = 0.02
